In [3]:
#!/usr/bin/env python3
import pandas as pd
from geopy.geocoders import Bing
import csv  
import json   
import warnings
warnings.filterwarnings('ignore')
bing_key = 'ginITT5ALcrOnA4aMoH0~_vGQBu-_tnKr6FYRj6U5nA~Ar9conblprRVoeaB4uH4PKMzcs0BfrUJ5LbspSyDn04UqCpCOfsm567DJjUw-PY6'

ModuleNotFoundError: No module named 'geopy'

In [5]:
def get_geocode(row):
    geolocator = Bing(bing_key, timeout=7)
    location = geolocator.geocode(row)
    if location is None:
        print("NOT FOUND")
        return "NOT FOUND"
    else:
        if location.raw["address"]["countryRegion"] == "Chile":
            r = str(location.latitude) + ", " + str(location.longitude)
            print(r)
            return r
        else:
            print("MISFORMED")
            return "MISFORMED"
        
def geocode(tabla):
    if(tabla == 'LEED.csv'):
        a = pd.read_csv(tabla,sep=',')
        b = a.copy()

        c = b[(b['ProjectName'] != 'Confidential') & (b['Street'].notnull()) & (b['Country'] == 'CL')]

        c['Pais'] = 'Chile'
        c['direccion'] = c[['Street', 'City', 'State', 'Pais']].apply(lambda x: ', '.join(x), axis=1)    
        # Geocoding #
        c['Coordenadas'] = c['direccion'].apply(get_geocode)

        # Lat and Lon
        lat = lambda row: row.split(',')[0].strip()
        lon = lambda row: row.split(',')[-1].strip()
        c['lat'] = c['Coordenadas'].map(lat)
        c['lon'] = c['Coordenadas'].map(lon)
        c.drop(labels=['Coordenadas'],axis=1,inplace=True)

        # Save to CSV
        n = 'data/coor_' + tabla.split('.')[0] + '.csv'
        c.to_csv(n, sep=',', encoding='utf-8')
        print(tabla + ' Done! ')
        
    elif(tabla == 'CES.csv'):
        a = pd.read_csv(tabla,sep=',')
        b = a.copy()
        c = b[(b['Dirección'].notnull())]
        c['Pais'] = 'Chile'
        c['direccion'] = c[['Dirección', 'Comuna', 'UBICACION', 'Pais']].apply(lambda x: ', '.join(x), axis=1)
        
        # Geocoding #
        c['Coordenadas'] = c['direccion'].apply(get_geocode)

        # Lat and Lon
        lat = lambda row: row.split(',')[0].strip()
        lon = lambda row: row.split(',')[-1].strip()
        c['lat'] = c['Coordenadas'].map(lat)
        c['lon'] = c['Coordenadas'].map(lon)
        c.drop(labels=['Coordenadas'],axis=1,inplace=True)

        # Save to CSV
        n = 'data/coor_' + tabla.split('.')[0] + '.csv'
        c.to_csv(n, sep=',', encoding='utf-8')
        print(tabla + ' Done! ')  
        
#     # Open the CSV  
#     f = open(n, 'rU')  
#     # Change each fieldname to the appropriate field name. I know, so difficult.  
#     reader = csv.DictReader(f, fieldnames = ('ID','Isconfidential','ProjectName','Street','City','State','Zipcode','Country','LEEDSystemVersionDisplayName','Sistema','PointsAchieved','CertLevel','CertDate','Mes certificación','Año' 'certificación','IsCertified','OwnerTypes','GrossSqFoot','mt2','TotalPropArea','ProjectTypes','Categoría','OwnerOrganization','RegistrationDate','Mes registro','Año registro','Pais','direccion','lat','lon'))  
    
#     # Parse the CSV into JSON  
#     out = json.dumps( [ row for row in reader ] )  
#     print("JSON parsed!")  
    
#     # Save the JSON  
#     f = open('leed.json', 'w')  
#     f.write(out)  
    
#     print("JSON saved!") 

In [6]:
print("ubicaciones LEED")
geocode('LEED.csv')

print ("ubicaciones CES")
geocode('CES.csv')

ubicaciones LEED


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


NameError: name 'Bing' is not defined

In [ ]:
import csv, json
from geojson import Feature, FeatureCollection, Point

In [ ]:
features = []
with open('data/coor_LEED.csv', newline='') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    for lat, lon in reader:
        lat, lon = map(float, (lat, lon))features.append(
            Feature(
                geometry = Point((lon, lat)),
                properties = {
                    'weather': 'weather',
                    'temp': 'temp'
                }
            )
        )

collection = FeatureCollection(features)
with open('data/coor_LEED.geojson', "w") as f:
    f.write('%s' % collection)